In [1]:
from symspellpy.symspellpy import SymSpell, Verbosity
from time import time
import pandas as pd
import numpy as np
import pickle
import random
import spacy
import json
import os
import re

from sklearn.feature_extraction.text import TfidfVectorizer

import keras
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

Using TensorFlow backend.


# ML Model Building

In [2]:
def batch_generator_shuffle(X_data, y_data, batch_size):
    samples_per_epoch = X_data.shape[0]
    number_of_batches = samples_per_epoch/batch_size
    counter=0
    index = np.arange(np.shape(y_data)[0])
    np.random.shuffle(index)
    while 1:
        index_batch = index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X_data[index_batch,:].toarray()
        y_batch = y_data[y_data.index[index_batch]]
        counter += 1
        yield X_batch,y_batch
        if (counter > number_of_batches):
            np.random.shuffle(index)
            counter=0

In [3]:
df_train = pd.read_csv('train_sets/TF-IDF_NNLR3_train.csv',encoding='utf-8-sig',index_col='tweetid')
df_test = pd.read_csv('test_sets/TF-IDF_MultinomialNB_test.csv',encoding='utf-8-sig',index_col='tweetid')
df_validation = pd.read_csv('validation_sets/validation_corrected.csv',encoding='utf-8-sig',index_col='id')

In [4]:
df_train = df_train.sample(frac=1,random_state=24)
df_train.head()

,clean_content_no,lemma_content_no,stemm_content_no,sentiments
tweetid,,,,
1134808220537630722,hace una semana estaba lista para mi prefiesta...,hacer uno semana estar listo parir mi prefiest...,hac una seman estab list par mi prefiest ahori...,N
1131548672246730752,un chalao en canta como colo de ese chalao soy yo,uno chalao en cantar comer colo de ese chalao ...,un chala en cant com col de ese chala soy yo,N
1131940623769055232,mira mami sin cruda,mirar mami sin crudo,mir mami sin crud,P
171566521334771712,en la protesta vecinal contra el cierre del am...,en lo protestar vecinal contra el cerrar del a...,en la protest vecinal contr el cierr del ambul...,N
1137017243428982784,delidesayunos time con la cálidas y sabor que ...,delidesayunos timar con lo cálido y sabor que ...,delidesayun tim con la cal y sabor que te mere...,N


In [5]:
vectorizer = TfidfVectorizer(max_df=0.5,min_df=7,use_idf=False,norm='l2',max_features=11000,ngram_range=(1, 2),stop_words=None)

seed = 7
np.random.seed(seed)

df_train.reset_index(inplace=True)
df_test.reset_index(inplace=True)

X_train,y_train = df_train['lemma_content_no'],pd.Series([1 if i=='P' else 0 for i in df_train['sentiments']],index = df_train.index)
X_test,y_test = df_test['lemma_content_no'],pd.Series([1 if i=='P' else 0 for i in df_test['value']],index = df_test.index)

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [6]:
model = Sequential()
custom_adam = keras.optimizers.Adam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
model.add(Dense(128, activation='relu', input_dim=11000))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=custom_adam,
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit_generator(generator=batch_generator_shuffle(X_train, y_train, 32),
                    epochs=2, validation_data=(X_test, y_test),
                    steps_per_epoch=X_train.shape[0]/32)

W0829 11:38:43.705867 11732 deprecation_wrapper.py:119] From C:\anaconda3\envs\benja\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0829 11:38:44.156119 11732 deprecation_wrapper.py:119] From C:\anaconda3\envs\benja\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0829 11:38:44.177107 11732 deprecation_wrapper.py:119] From C:\anaconda3\envs\benja\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0829 11:38:44.209089 11732 deprecation_wrapper.py:119] From C:\anaconda3\envs\benja\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0829 11:38:44.219083 11732 deprecation.py:506] Fr

Epoch 1/2


 549/3392 [===>..........................] - ETA: 1:06:04 - loss: 0.6927 - acc: 0.62 - ETA: 23:06 - loss: 0.6881 - acc: 0.5938 - ETA: 14:30 - loss: 0.6847 - acc: 0.61 - ETA: 9:32 - loss: 0.6902 - acc: 0.5508 - ETA: 7:17 - loss: 0.6902 - acc: 0.545 - ETA: 6:00 - loss: 0.6851 - acc: 0.553 - ETA: 5:11 - loss: 0.6857 - acc: 0.553 - ETA: 4:36 - loss: 0.6829 - acc: 0.554 - ETA: 4:10 - loss: 0.6801 - acc: 0.565 - ETA: 3:50 - loss: 0.6778 - acc: 0.572 - ETA: 3:34 - loss: 0.6751 - acc: 0.578 - ETA: 3:21 - loss: 0.6734 - acc: 0.578 - ETA: 3:10 - loss: 0.6688 - acc: 0.591 - ETA: 3:01 - loss: 0.6647 - acc: 0.602 - ETA: 2:54 - loss: 0.6612 - acc: 0.609 - ETA: 2:47 - loss: 0.6567 - acc: 0.610 - ETA: 2:41 - loss: 0.6539 - acc: 0.614 - ETA: 2:36 - loss: 0.6522 - acc: 0.619 - ETA: 2:31 - loss: 0.6487 - acc: 0.623 - ETA: 2:27 - loss: 0.6474 - acc: 0.626 - ETA: 2:24 - loss: 0.6487 - acc: 0.625 - ETA: 2:21 - loss: 0.6456 - acc: 0.630 - ETA: 2:19 - loss: 0.6446 - acc: 0.631 - ETA: 2:16 - loss: 0.6435 - acc

1120/3392 [========>.....................] - ETA: 1:14 - loss: 0.5607 - acc: 0.708 - ETA: 1:14 - loss: 0.5604 - acc: 0.709 - ETA: 1:14 - loss: 0.5607 - acc: 0.709 - ETA: 1:14 - loss: 0.5605 - acc: 0.709 - ETA: 1:14 - loss: 0.5603 - acc: 0.709 - ETA: 1:14 - loss: 0.5597 - acc: 0.709 - ETA: 1:14 - loss: 0.5592 - acc: 0.710 - ETA: 1:14 - loss: 0.5594 - acc: 0.710 - ETA: 1:14 - loss: 0.5590 - acc: 0.710 - ETA: 1:14 - loss: 0.5586 - acc: 0.710 - ETA: 1:13 - loss: 0.5578 - acc: 0.711 - ETA: 1:13 - loss: 0.5578 - acc: 0.711 - ETA: 1:13 - loss: 0.5576 - acc: 0.711 - ETA: 1:13 - loss: 0.5575 - acc: 0.711 - ETA: 1:13 - loss: 0.5577 - acc: 0.711 - ETA: 1:13 - loss: 0.5572 - acc: 0.711 - ETA: 1:13 - loss: 0.5569 - acc: 0.711 - ETA: 1:13 - loss: 0.5567 - acc: 0.711 - ETA: 1:13 - loss: 0.5567 - acc: 0.711 - ETA: 1:12 - loss: 0.5565 - acc: 0.712 - ETA: 1:12 - loss: 0.5562 - acc: 0.712 - ETA: 1:12 - loss: 0.5561 - acc: 0.712 - ETA: 1:12 - loss: 0.5558 - acc: 0.712 - ETA: 1:12 - loss: 0.5556 - acc: 0.7

1676/3392 [=============>................] - ETA: 57s - loss: 0.5367 - acc: 0.72 - ETA: 56s - loss: 0.5365 - acc: 0.72 - ETA: 56s - loss: 0.5366 - acc: 0.72 - ETA: 56s - loss: 0.5366 - acc: 0.72 - ETA: 56s - loss: 0.5365 - acc: 0.72 - ETA: 56s - loss: 0.5364 - acc: 0.72 - ETA: 56s - loss: 0.5364 - acc: 0.72 - ETA: 56s - loss: 0.5364 - acc: 0.72 - ETA: 56s - loss: 0.5364 - acc: 0.72 - ETA: 56s - loss: 0.5363 - acc: 0.72 - ETA: 56s - loss: 0.5363 - acc: 0.72 - ETA: 56s - loss: 0.5361 - acc: 0.72 - ETA: 56s - loss: 0.5361 - acc: 0.72 - ETA: 56s - loss: 0.5360 - acc: 0.72 - ETA: 56s - loss: 0.5359 - acc: 0.72 - ETA: 55s - loss: 0.5358 - acc: 0.72 - ETA: 55s - loss: 0.5359 - acc: 0.72 - ETA: 55s - loss: 0.5357 - acc: 0.72 - ETA: 55s - loss: 0.5357 - acc: 0.72 - ETA: 55s - loss: 0.5357 - acc: 0.72 - ETA: 55s - loss: 0.5356 - acc: 0.72 - ETA: 55s - loss: 0.5356 - acc: 0.72 - ETA: 55s - loss: 0.5355 - acc: 0.72 - ETA: 55s - loss: 0.5354 - acc: 0.72 - ETA: 55s - loss: 0.5354 - acc: 0.72 - ETA: 

...

1195/3392 [=========>....................] - ETA: 1:05 - loss: 0.4121 - acc: 0.806 - ETA: 1:05 - loss: 0.4121 - acc: 0.806 - ETA: 1:05 - loss: 0.4122 - acc: 0.806 - ETA: 1:05 - loss: 0.4123 - acc: 0.806 - ETA: 1:05 - loss: 0.4125 - acc: 0.805 - ETA: 1:04 - loss: 0.4124 - acc: 0.805 - ETA: 1:04 - loss: 0.4123 - acc: 0.805 - ETA: 1:04 - loss: 0.4122 - acc: 0.806 - ETA: 1:04 - loss: 0.4123 - acc: 0.805 - ETA: 1:04 - loss: 0.4118 - acc: 0.806 - ETA: 1:04 - loss: 0.4118 - acc: 0.806 - ETA: 1:04 - loss: 0.4117 - acc: 0.805 - ETA: 1:04 - loss: 0.4116 - acc: 0.805 - ETA: 1:04 - loss: 0.4117 - acc: 0.805 - ETA: 1:04 - loss: 0.4114 - acc: 0.805 - ETA: 1:04 - loss: 0.4114 - acc: 0.805 - ETA: 1:04 - loss: 0.4113 - acc: 0.806 - ETA: 1:04 - loss: 0.4119 - acc: 0.805 - ETA: 1:04 - loss: 0.4118 - acc: 0.805 - ETA: 1:04 - loss: 0.4118 - acc: 0.805 - ETA: 1:04 - loss: 0.4120 - acc: 0.805 - ETA: 1:04 - loss: 0.4119 - acc: 0.805 - ETA: 1:03 - loss: 0.4123 - acc: 0.804 - ETA: 1:03 - loss: 0.4127 - acc: 0.8

1827/3392 [===============>..............] - ETA: 50s - loss: 0.4172 - acc: 0.80 - ETA: 50s - loss: 0.4172 - acc: 0.80 - ETA: 50s - loss: 0.4170 - acc: 0.80 - ETA: 50s - loss: 0.4170 - acc: 0.80 - ETA: 50s - loss: 0.4168 - acc: 0.80 - ETA: 50s - loss: 0.4167 - acc: 0.80 - ETA: 50s - loss: 0.4169 - acc: 0.80 - ETA: 50s - loss: 0.4169 - acc: 0.80 - ETA: 50s - loss: 0.4169 - acc: 0.80 - ETA: 50s - loss: 0.4171 - acc: 0.80 - ETA: 49s - loss: 0.4171 - acc: 0.80 - ETA: 49s - loss: 0.4173 - acc: 0.80 - ETA: 49s - loss: 0.4171 - acc: 0.80 - ETA: 49s - loss: 0.4170 - acc: 0.80 - ETA: 49s - loss: 0.4172 - acc: 0.80 - ETA: 49s - loss: 0.4171 - acc: 0.80 - ETA: 49s - loss: 0.4171 - acc: 0.80 - ETA: 49s - loss: 0.4169 - acc: 0.80 - ETA: 49s - loss: 0.4170 - acc: 0.80 - ETA: 49s - loss: 0.4169 - acc: 0.80 - ETA: 49s - loss: 0.4170 - acc: 0.80 - ETA: 49s - loss: 0.4170 - acc: 0.80 - ETA: 49s - loss: 0.4170 - acc: 0.80 - ETA: 49s - loss: 0.4172 - acc: 0.80 - ETA: 48s - loss: 0.4172 - acc: 0.80 - ETA: 

2459/3392 [====================>.........] - ETA: 35s - loss: 0.4208 - acc: 0.80 - ETA: 35s - loss: 0.4209 - acc: 0.80 - ETA: 35s - loss: 0.4210 - acc: 0.80 - ETA: 35s - loss: 0.4209 - acc: 0.80 - ETA: 35s - loss: 0.4210 - acc: 0.80 - ETA: 35s - loss: 0.4209 - acc: 0.80 - ETA: 35s - loss: 0.4209 - acc: 0.80 - ETA: 35s - loss: 0.4210 - acc: 0.80 - ETA: 35s - loss: 0.4211 - acc: 0.80 - ETA: 35s - loss: 0.4210 - acc: 0.80 - ETA: 35s - loss: 0.4212 - acc: 0.80 - ETA: 35s - loss: 0.4212 - acc: 0.80 - ETA: 35s - loss: 0.4213 - acc: 0.80 - ETA: 35s - loss: 0.4215 - acc: 0.80 - ETA: 35s - loss: 0.4216 - acc: 0.80 - ETA: 35s - loss: 0.4215 - acc: 0.80 - ETA: 35s - loss: 0.4216 - acc: 0.80 - ETA: 34s - loss: 0.4215 - acc: 0.80 - ETA: 34s - loss: 0.4215 - acc: 0.80 - ETA: 34s - loss: 0.4216 - acc: 0.80 - ETA: 34s - loss: 0.4217 - acc: 0.80 - ETA: 34s - loss: 0.4218 - acc: 0.80 - ETA: 34s - loss: 0.4218 - acc: 0.80 - ETA: 34s - loss: 0.4217 - acc: 0.80 - ETA: 34s - loss: 0.4217 - acc: 0.80 - ETA: 

3075/3392 [==========================>...] - ETA: 21s - loss: 0.4262 - acc: 0.79 - ETA: 21s - loss: 0.4262 - acc: 0.79 - ETA: 21s - loss: 0.4262 - acc: 0.79 - ETA: 21s - loss: 0.4262 - acc: 0.79 - ETA: 21s - loss: 0.4263 - acc: 0.79 - ETA: 21s - loss: 0.4264 - acc: 0.79 - ETA: 20s - loss: 0.4264 - acc: 0.79 - ETA: 20s - loss: 0.4265 - acc: 0.79 - ETA: 20s - loss: 0.4265 - acc: 0.79 - ETA: 20s - loss: 0.4265 - acc: 0.79 - ETA: 20s - loss: 0.4265 - acc: 0.79 - ETA: 20s - loss: 0.4266 - acc: 0.79 - ETA: 20s - loss: 0.4266 - acc: 0.79 - ETA: 20s - loss: 0.4266 - acc: 0.79 - ETA: 20s - loss: 0.4265 - acc: 0.79 - ETA: 20s - loss: 0.4266 - acc: 0.79 - ETA: 20s - loss: 0.4267 - acc: 0.79 - ETA: 20s - loss: 0.4267 - acc: 0.79 - ETA: 20s - loss: 0.4267 - acc: 0.79 - ETA: 20s - loss: 0.4267 - acc: 0.79 - ETA: 20s - loss: 0.4267 - acc: 0.79 - ETA: 20s - loss: 0.4268 - acc: 0.79 - ETA: 19s - loss: 0.4267 - acc: 0.79 - ETA: 19s - loss: 0.4268 - acc: 0.79 - ETA: 19s - loss: 0.4268 - acc: 0.79 - ETA: 

3393/3392 [==============================] - ETA: 7s - loss: 0.4308 - acc: 0.795 - ETA: 7s - loss: 0.4308 - acc: 0.795 - ETA: 7s - loss: 0.4309 - acc: 0.795 - ETA: 7s - loss: 0.4309 - acc: 0.795 - ETA: 6s - loss: 0.4309 - acc: 0.795 - ETA: 6s - loss: 0.4310 - acc: 0.795 - ETA: 6s - loss: 0.4310 - acc: 0.795 - ETA: 6s - loss: 0.4310 - acc: 0.795 - ETA: 6s - loss: 0.4310 - acc: 0.795 - ETA: 6s - loss: 0.4310 - acc: 0.795 - ETA: 6s - loss: 0.4310 - acc: 0.795 - ETA: 6s - loss: 0.4310 - acc: 0.795 - ETA: 6s - loss: 0.4310 - acc: 0.795 - ETA: 6s - loss: 0.4310 - acc: 0.795 - ETA: 6s - loss: 0.4310 - acc: 0.795 - ETA: 6s - loss: 0.4310 - acc: 0.795 - ETA: 6s - loss: 0.4310 - acc: 0.795 - ETA: 6s - loss: 0.4309 - acc: 0.795 - ETA: 5s - loss: 0.4310 - acc: 0.795 - ETA: 5s - loss: 0.4310 - acc: 0.795 - ETA: 5s - loss: 0.4310 - acc: 0.795 - ETA: 5s - loss: 0.4310 - acc: 0.795 - ETA: 5s - loss: 0.4310 - acc: 0.795 - ETA: 5s - loss: 0.4311 - acc: 0.795 - ETA: 5s - loss: 0.4311 - acc: 0.795 - ETA: 

In [7]:
X_validation =  df_validation['lemma_content_no']
X_validation = vectorizer.transform(X_validation)

df_validation['NNLR3_lemma_proba'] = model.predict(X_validation)
df_validation['NNLR3_lemma_value'] = ['P' if t > 0.66 else 'NEU' if 0.66>t>0.56 else 'N' for t in df_validation['NNLR3_lemma_proba']]
df_validation.head()

,sesion id,evento,texto,actividad,respuesta,corrected_content_no,corrected_respuesta,lemma_content_no,lemma_content,stemm_content_no,stemm_content,NNLR3_lemma_proba,NNLR3_lemma_value
id,,,,,,,,,,,,,
1239.0,210,Conflicto_La_Vida,¿Te gustaría contarme más sobre esto?,Problematica,"nada, simplemente desepcionado de como.se.dan ...",nada simplemente decepcionado de como las cosas,simplemente decepcionado,nadar simplemente decepcionar de comer los coser,simplemente decepcionar,nad simplement decepcion de com las cos,simplement decepcion,0.206171,N
1245.0,210,Debate_Creencia_3_Bien,¿Qué otra explicación le darías?,Explicacion,pues ver lo positivo de ello,pues ver lo positivo de ello,positivo,pues ver el positivo de él,positivo,pues ver lo posit de ello,posit,0.738438,P
1248.0,210,Compartir_Aprendido,¿Qué aprendiste de lo qué abordamos hoy?,Aprendizaje,darme un tiempo para refelxionar y no nada mas...,darme un tiempo para reflexionar y no nada mas...,darme reflexionar y irme a i precio o pensamiento,darme uno tiempo parir reflexionar y no nadar ...,darme reflexionar y irme a i preciar o pensami...,darm un tiemp par reflexion y no nad mas irme ...,darm reflexion y irme a i preci o pensamient,0.770850,P
1329.0,226,No_Obstaculo_Detectado_2,¿Deseas compartirme cual fue tu obstáculo?,Obstaculo_Actividad,planeo realizarla por la tarde del día,planeo realizara por la tarde del día,planeo realizara,planear realizar por lo tardar del día,planear realizar,plane realiz por la tard del dia,plane realiz,0.956102,P
1345.0,255,Conflicto_Los_Demas,¿Te gustaría contarme más sobre esto?,Problematica,las personas a quienes atiendo me hacen enojar...,las personas a quienes atiendo me hacen enojar...,personas a atiendo enojar entienden instruccio...,los personar a quien atender me hacer enojar p...,personar a atender enojar entender instrucción...,las person a quien atiend me hac enoj porqu no...,person a atiend enoj entiend instruccion y quier,0.198364,N


In [8]:
# with open('models/vectorizer_TF-IDF.pkl', 'wb') as f:
#     pickle.dump(vectorizer, f)

# model_s_3.save('NN_models/TF-IDF_NNLR3_lemma.h5')

# Web Service

In [9]:
def clean_text(text):

    text = text.lower()

    text = re.sub(r'\bq\b|\bk\b', 'que', text) # replace q or x with que
    text = re.sub(r'\bd\b', 'de', text) # replace d with de
    text = re.sub(r'\bx\b', 'por', text) # replace x with por
    text = re.sub(r'\btmb\b', 'también', text) # replace tmb with tambien
    text = re.sub(r'\bbb\b', 'bebé', text) # replace bb with bebe

    duplicates = re.compile(r'([^(c,l,n,r)L0])\1{1,}')
    double_clnr = re.compile(r"(.)\1{2,}")
    while duplicates.search(text)!=None:
        text = text.replace(duplicates.search(text).group(),duplicates.search(text).group()[0]) #remove multiple letters
    text = double_clnr.sub(r"\1\1", text) #except double c, l, n, and r

    text = re.sub(r'([ja]{5,}|[je]{5,}|[ji]{5,}|[ha]{5,}|[he]{5,})', 'jaja', text)  # remove dirty laughs

    text = re.sub(r'(\.|,|:|;|!|\?|\[|\]|\(|\))[A-Za-z0-9]+', ' ', text)  # replace simbols between words with spaces
    text = re.sub(r'\d+', '', text) #remove numbers

    text = re.sub(r'[%s]' % re.escape("""¿¡!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~…"""), '', text)  # remove punctuations
    text = re.sub(r'\b[^aeyou]\b', ' ', text) # remove single char
    text = re.sub('\s+', ' ', text)  # remove extra whitespace


    text = text.encode('latin', 'ignore').decode('latin')

    max_edit_distance_lookup = 3
    text = sym_spell.lookup_compound(text,max_edit_distance_lookup)[0].term


    tokens= nlp(u""+text)
    new_text= ' '.join([t.lemma_ for t in tokens])


    return [new_text]

In [10]:
nlp = spacy.load("es_core_news_md")

In [11]:
sym_spell = SymSpell(
    max_dictionary_edit_distance=3,
    prefix_length=7,
    count_threshold=1,
    compact_level=5,
)

sym_spell.load_dictionary(corpus='resources/es_real_freq_full.txt',term_index=0,count_index=1,encoding='utf-8')

True

In [12]:
text_examples = ["nada, simplemente desepcionado de como.se.dan las cosas",
"las personas a quienes atiendo me hacen enojar porque no entienden mis instrucciones y hacen lo que quieren",
"claro! mi problemática radica en que se me dificulta entablar relación con mis compañeros de trabajo. Motivo por el cual el ambiente laboral es tenso",
"me molesta la falta de compromiso de un miembro de mi familia para ayudar en casa",
"me quede sin trabajo hace días",
"me hago ilusiones con personas muy rapido",
"que ya no pude terminar mis estudios superiores y no he podido encontrar  jn mejor empleo",
"voy a comezar a vivir con mi esposo, no tenemos mucha estabilidad economica, es injusto trabajar tanto y aun asi tener problemas economicos",
"la pareja que tenia me mintio y engaño mucho",
"ayer fue cumpleaños de mi novio y lo mande felicitar por fb y borro la felicitacion y mi foto y eso me molesto mucho",
"hoy me robaron dinero de un tramite de la oficina eso me causo preocupacion y tuve mucho trabajo, me acelere mucho y con la tension del problema... me causo estres y al final de la jornada cansancio y un poco de ansiedad",
"No me esfuerzo lo suficiente",
"fracaso de pareja",
"no puedo terminar los trabajos a  tiempo",
"fallecio una persona muy querida, era como si fuera parte de mi familia ... esto me hace sentir triste",
"almejo se va a otros lados en lugar de estar en el proyecto del eneit",
"me siento tonta",
"un posible trabajo me poner en la necesidad de cambiar de residencia",
"la verdad ni yo se que es lo que pasa,.de repente esroy.bien y de rato unos 'bajones' feos",
"siento que voy a reprobar",
"que no me atiende",
"me siento insegura al abordar una conversación me da miedo hablar algunas cosas",
"me molesta mucho que mi esposo se tome las cosas con tanta calma... que sea tan decidioso... que tenga que estarlo presionando para que haga las cosas",
"mi hijo se va a regresar con su padre porque quiere estudiar allá, yo me siento impotente enojada porque no logre los planes de cuando llegamos aquí y ahora el se va se que es para superarse pero creo que no di el 100 con él",
"Mi novia no me ayuda con las labores del hogar",
"no es justo que yo trabaje esforzandome mucho para terminar con mas rapidez... y las compañeras una sea muy lenta y la otra sea tan conchuda eso mr irrita bastante que nos le llamen la atencion me da mucho coraje",
"estoy como aburrida..solo quiero irme a acostar..nada me llama la atencion",
"mesiento inseguro esto mebuelbe distraido esto metrae prpblemas en mi trabajo",
"planeo realizarla por la tarde del día",
"nada simplemente aceptar que tengo problemas psicológicos",
"No tuve tiempo de hacerla",
"no me di el tiempo para hacerlo",
"pienso realizarla un rato mas",
"falta de tiempo",
"no me organizo con mis tiempos, me gana el cansancio o la flojera",
"cambie el dia porque tube hoy cosas q hacer..espero que mañana se pueda",
"Aun no termina el día",
"si la realice solo que algunas veces no se como darle la respuesta.la actividad es con mi hijo ayer le dije que lo quiero mucho y que es lo mas valioso que tengo, hoy le dije que termine sus materias que yo siempre lo voy apoyar",
"sali a convivir con mis hijos",
"No me puse de acuerdo con la amiga para salir",
"estaba lloviendo y tuve que regresar mas rapido",
"no necesite ayuda",
"La lluvia",
"Falta de tiempo",
"Cambio de planes",
"No hubo obstáculo",
"pues ver lo positivo de ello",
"considere mas opciones",
"no lo sé",
"que busque empleo donde se sienta mejor",
"que es muy valioso que puede salir adelante solo",
"que le echara ganas que nadie nace enseñado",
"no te preocupes todo va a mejorar, asi es como comienzan las parejas pero es parte del matrimonio, no te desanimes echenle  ganas",
"que buscara otra y se olvidara de ella",
"que vale mucho,,,pero no ha llegado la persona correcta",
"si tu pareja te ignora es porque no le importas",
"abraza tu ansiedad no pasa nada, estas en tx todo va fluyendo poco a poco respira... piensa en detectar que situacion te pone asi... disfruta cada momento relajate",
"No te excuses",
"que no sea como yo",
"etres, presion",
"no tengo explicación",
"que tuviera comunicacion",
"que el no ya es seguro hay que buscar el si",
"ten paciencia poco a poco... todo saldra bien",
"que se vea en mi y no cometa los mismos errores pues no se beneficia en nada y se perjudica a los hijos",
"Llegar a un acuerdo para repartir las labores",
"nose",
"no se..no tengo ganas de dar ninguna explicación",
"necesito integrarme mas a las actividades en grupo y quitar las barreras que implemento para que no me conozcan",
"porque no puedo desempeñarme en cualquier área de trabajo",
"que dificil es comenzar una vida juntos, muchos obstaculos",
"tengo miedo de tener crisis de ansiedad",
"miedo a sentir ansiedad",
"que no podremos salir adelante con tantos gastos... yo no puedo sola",
"no he luchado lo suficiente para darles a mis hijos lo que necesitan y la atención que merecen",
"Es necesario que me ayude con algunas labores",
"darme un tiempo para refelxionar y no nada mas irme a la primera imprecion o pensamiento.",
"no centrarme solo en una solución, sino ver mas opciones y pensar de manera diferente",
"que el preocuparme por las acciones de los otros altera mi comportamiento. necesito centrarme en mí y mis actividades para que mi bienestar no se vea afectado.",
"No tomar importancia de las acciones de los demás",
"no pensar negativamente",
"a valorarme mas y aceptar las cosas",
"que debo aceptarme tal y cual soy",
"mmmm no tener un pensamiento o vision tan cuadrada... aprender a observar y tomar calma ante todo",
"que no debo sufrir por acciones ajenas",
"que no estoy sola....y que valgo  mucho",
"pues es verdad una persona no es todo el mundo hay personas q de verdad nos aprecian..no podemos cambiar la forma de ser de nadie..solo la nuestra",
"No pienses,actua",
"que valgo por mi pefsona",
"alrender a escuchar y respetar las opiniones de los demas",
"las cosas en que fallo no determinan quien soy",
"que los pensamientos me hacen sentir mal",
"que me debo ocupar mas de mi",
"sigo molesta",
"que no debo permitir que los comentarios de mis hijos me lastimen ya que para mi son mi prioridad",
"Llegar a un acuerdo",
"seguir adelante",
"respirar pensar con mas calma y mas friamente sobre el asunto para poder decidir que hacer",
"si no me gusta la vida q estoy viviendo que cambie un poco que la mejore"]

In [13]:
my_vectorizer = pickle.load(open('models/vectorizer_TF-IDF.pkl','rb'))
best_model = load_model('NN_models/TF-IDF_NNLR3_lemma.h5')

In [14]:
# my_text = 'Tengo muchas deudas en el banco. Si no pago me meteran a la carcel y tendre que alejarme de mis hisjo'

my_text = "".join(random.sample(text_examples,1))

start = time()
print(f"\nInput text: {my_text}")

my_text = my_vectorizer.transform(clean_text(my_text))

score = best_model.predict(my_text)[0]

print(f"Probability: {score}")
emotion,intensity,decay = (("Happy_For",120,3) if score>=0.8 
                           else ("Joy",100,6) if 0.66<=score<0.8
                           else ("Neutral",0,6) if 0.56<=score<0.66
                           else ("Pity",55,6) if 0.26<=score<0.56
                           else ("Pity",80,3))
print(f"Emotion: {emotion}\nIntensity: {intensity}\nDecay: {decay}")

print(f"Process took {time()-start} seconds to finish\n")


Input text: no me di el tiempo para hacerlo
Probability: [0.28821504]
Emotion: Pity
Intensity: 55
Decay: 6
Process took 0.3831632137298584 seconds to finish

